# Divergence Vectors on Corridor Experiment

In this notebook, we run various simulations on the corridor experimetn using both HL and ORCA behaviuors.

We use divergence vectors to see if these are able to distinguish between both.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl

import os

from navground import core, sim

import perdiver.perdiver as perdiver
from perdiver.distances import *

plots_dir = "plots"
os.makedirs(plots_dir, exist_ok=True)

We run a few times the corridor experiment.

In [ ]:
length = 8.0
num_steps = 400
width=1.0
num_agents = 31 # 38
num_runs = 10
behaviour_list = ["ORCA", "HL", "HRVO", "Dummy"]
color_behaviour = {}
for i, behaviour in enumerate(behaviour_list):
    color_behaviour[behaviour] = mpl.colormaps["Set1"](i / (len(behaviour_list) +1))
for behaviour in behaviour_list:
    path=f"experiment_matching_finer_{behaviour}.h5"
    yaml = f"""
    steps: {num_steps}
    time_step: 0.1
    record_pose: true
    record_twist: true
    record_collisions: true
    record_deadlocks: true
    record_safety_violation: true
    record_efficacy: true
    runs: {num_runs}
    scenario:
      type: Corridor
      length: {length}
      width: {width} 
      groups:
        -
          type: thymio
          number: {num_agents}
          radius: 0.08
          control_period: 0.1
          speed_tolerance: 0.02
          kinematics:
            type: 2WDiff
            wheel_axis: 0.094
            max_speed: 0.166
          behavior:
            type: {behaviour}
            optimal_speed: 0.12
            horizon: 5.0
            safety_margin: 0.03
          state_estimation:
            type: Bounded
            range: 5.0
    """
    # optimal_speed: 
    #     sampler: normal
    #     mean: 0.2
    #     std_dev: 0.05
    # horizon: 5.0
    # safety_margin: 
    #     sampler: normal
    #     mean: 0.08
    #     std_dev: 0.05
    experiment = sim.load_experiment(yaml)
    experiment.run(keep=False, data_path=path)
    del experiment

We reload both experiment runs and save them into a dictionary of lists.

In [ ]:
runs = {}
# Reload HL simulation
for behaviour in behaviour_list:
    path = f"experiment_matching_finer_{behaviour}.h5"
    recorded_experiment = sim.RecordedExperiment(path)
    runs[behaviour] = recorded_experiment.runs

In [ ]:
# timestep_list = list(range(num_steps))
# variables_behaviour = {}
# for j, behaviour in enumerate(behaviour_list):
#     variables_list = []
#     for ridx in range(num_runs):
#         variables_run = []
#         ps = np.array(runs[behaviour][ridx].poses)
#         twists = np.array(runs[behaviour][ridx].twists)
#         for idx, step in enumerate(timestep_list):
#             X = ps[step]
#             vel_X = twists[step]
#             variables_run.append(np.hstack((np.mean(X, axis=0), np.std(X, axis=0), np.mean(vel_X, axis=0), np.std(vel_X, axis=0))))
#         # end for 
#         variables_list.append(np.array(variables_run).transpose())
#     # end for
#     variables_behaviour[behaviour] = np.array(variables_list)

In [ ]:
def distance_matrix_arrays_3D(A, axis=1):
    if len(A.shape)!=3:
        raise ValueError
    num_elements = A.shape[axis]
    Dist = np.zeros((num_elements, num_elements))
    for i in range(num_elements):
        for j in range(num_elements):
            Dist[i,j] = np.sqrt(np.sum((A.take(i, axis=axis)-A.take(j, axis=axis))**2))

    return Dist

Visualization

In [ ]:
from navground.sim.ui import WebUI
from navground.sim.notebook import notebook_view
from navground.sim.replay import RealTimeReplay

web_ui = WebUI(host='127.0.0.1', max_rate=-1)
await web_ui.prepare()

In [ ]:
notebook_view(width=300)

In [ ]:
rt_sim = RealTimeReplay(run=runs["Dummy"][0], factor=10, web_ui=web_ui)
await rt_sim.run()

In [ ]:
from sklearn.preprocessing import minmax_scale

# Parameters
initial_steps_list = range(100, 200, 10)
range_steps = 60
min_step = 5
# Plot divergence over time
divergences_list = []
fig, ax = plt.subplots(figsize=(6,2))
for behaviour in behaviour_list:
    for i_run in range(num_runs):
        # Load velocities and positions
        ps = np.array(runs[behaviour][i_run].poses)
        twists = np.array(runs[behaviour][i_run].twists)
        # Put together 
        # run_data = np.concatenate((ps, twists), axis=2)
        run_data = twists
        # Scale all variables to range from 0 to 1
        run_data_scaled = []
        for var_idx in range(run_data.shape[2]):
            scaled_var = np.transpose(minmax_scale(run_data[:,:,var_idx]))
            # scaled_var = np.transpose(run_data[:,:,var_idx]) # Omit scaling
            run_data_scaled.append(scaled_var)
        
        run_data_scaled = np.array(run_data_scaled)
        # Reorder array, putting agents on first coordinate
        run_data_ready = []
        for agent_id in range(run_data_scaled.shape[1]):
            run_data_ready.append(run_data_scaled[:, agent_id, :])
        run_data_ready = np.array(run_data_ready)
        # Compute divergence sums for run
        divergence_sums = []
        initial_step = initial_steps_list[0]
        prev_step_list = range(initial_step, initial_step + range_steps, min_step)
        for initial_step in initial_steps_list[1:]:
            step_list = range(initial_step, initial_step + range_steps, min_step)
            Dist_X = distance_matrix_arrays_3D(run_data_ready[:,:,prev_step_list ], axis=0)
            Dist_Y = distance_matrix_arrays_3D(run_data_ready[:,:,step_list] , axis=0)
            match_diagram = perdiver.get_matching_diagram(Dist_X, Dist_Y)
            divergence_sums.append(np.sum(np.abs(match_diagram[:,1]-match_diagram[:,0])))
            prev_step_list = step_list
        # Plot divergence sums
        ax.plot(initial_steps_list[:-1], divergence_sums, color=color_behaviour[behaviour], label=behaviour)
        divergences_list.append(divergence_sums)

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.3,0),  ncol=len(behaviour_list))
plt.tight_layout()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
Y = pca.fit_transform(divergences_list)
fig, ax = plt.subplots(figsize=(4,4))
for idx, behaviour in enumerate(behaviour_list):
    ax.scatter(Y[idx*num_runs:(idx+1)*num_runs,0], Y[idx*num_runs:(idx+1)*num_runs,1], color=color_behaviour[behaviour], label=behaviour)

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.3,0),  ncol=len(behaviour_list))
plt.tight_layout()

Use the following function from Navground documentation.

In [ ]:
import numpy as np
import pandas as pd


def count_deadlocks(deadlock_time, final_time):
    is_deadlocked = np.logical_and(deadlock_time > 0, deadlock_time < (final_time - 5.0))
    return sum(is_deadlocked)

def extract_data(experiment):
    collisions = []
    deadlocks = []
    efficacy = []
    sms = []
    seeds = []
    for i, run in experiment.runs.items():
        world = run.world
        sm = np.unique([agent.behavior.safety_margin for agent in world.agents])
        sms += list(sm)
        seeds.append(run.seed)
        final_time = run.world.time
        deadlocks.append(count_deadlocks(np.array(run.deadlocks), final_time))
        collisions.append(len(run.collisions))
        efficacy.append(np.array(run.efficacy).mean())

    df = pd.DataFrame({
        'seeds': seeds,
        'safety_margin': sms,
        'deadlocks': deadlocks,
        'collisions': collisions,
        'efficacy': efficacy})
    df['safe'] = (df.collisions == 0).astype(int)
    df['fluid'] = (df.deadlocks == 0).astype(int)
    df['ok'] = ((df.deadlocks == 0) & (df.collisions == 0)).astype(int)
    return df

In [ ]:
deadlocks_list = []
collisions_list = []
efficacy_list = []
for behaviour in behaviour_list:
    path = f"experiment_matching_finer_{behaviour}.h5"
    recorded_experiment = sim.RecordedExperiment(path)
    df = extract_data(recorded_experiment)
    collisions_list += list(df.collisions)
    deadlocks_list += list(df.deadlocks)
    efficacy_list += list(df.efficacy)

In [ ]:
fig, ax = plt.subplots(ncols=3)
# Plot PCA of points using divergences sums
for idx, behaviour in enumerate(behaviour_list):
    ax.scatter(Y[idx*num_runs:(idx+1)*num_runs,0], Y[idx*num_runs:(idx+1)*num_runs,1], color=color_behaviour[behaviour], label=behaviour)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
# Efficacy 
for idx, behaviour in enumerate(behaviour_list):
    ax.scatter(Y[idx*num_runs:(idx+1)*num_runs,0], Y[idx*num_runs:(idx+1)*num_runs,1], color=color_behaviour[behaviour], label=behaviour)


fig.legend(by_label.values(), by_label.keys(), loc=(0.3,0),  ncol=len(behaviour_list))
plt.tight_layout()

In [ ]:
efficacy_list

In [ ]:
mpl.colormaps["Greens"]

In [ ]:
points = np.vstack((np.array([efficacy_list]), np.array([collisions_list]))).transpose()

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
for idx, behaviour in enumerate(behaviour_list):
    points_beh = points[idx*num_runs:(idx+1)*num_runs]
    ax.scatter(points_beh[:,0], points_beh[:,1], color=color_behaviour[behaviour], label=behaviour)
# end for
ax.set_xlabel("efficacy")
ax.set_ylabel("collisions")
fig.legend(by_label.values(), by_label.keys(), loc=(0,0),  ncol=len(behaviour_list))
plt.tight_layout()